In [1]:
import time, sys
import web3 as w3; w3 = w3.Account
from objsize import get_deep_size as get_size
from eth_account.messages import encode_defunct as encode_msg
from hashlib import sha256

In [2]:
ph = lambda h: h if len(h)<16 else h[:16] + '...' + h[-3:] # print hash
rh = lambda: sha256(str(time.time()).encode()).hexdigest() # random hash

def unpack(o): return o.__dict__.values()

def addr2emoji(addr):
    offset = int(addr[0:4], 0)
    unicode = b'\U' + b'000' + str(hex(0x1F466+offset))[2:].encode()
    return unicode.decode('unicode_escape')

In [3]:
class Account:
    def __init__(self): 
        self.priv, self.pub = Account.keys()
        self.nonce = 0
        
    @staticmethod
    def keys():
        acc = w3.create()
        return acc.privateKey.hex(), acc.address
    
    def _sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
    
    def send(self, to, value): return self._sign(TX(self.pub, to.pub, value, self.nonce))
    
    def mine(self, block, diff=2, reward=50, attempts=1000):
        mblock   = MinebleBlock(*unpack(block), self.pub, diff, reward, block.size())
        mblock_b = bytes(mblock)
        nonce = 0
        for i in range(attempts):
            candidate   = mblock_b + str(nonce).encode()
            candidate_h = sha256(candidate).hexdigest()
            if candidate_h[:diff] == '0'*diff: return MinedBlock(*unpack(mblock), nonce)
            nonce += 1
            
    def __str__(self): return addr2emoji(self.pub)+' '+ph(self.pub)

In [4]:
acc1 = Account()
acc2 = Account()

print(acc1)
print(acc2)

🔎 0xA8Ff652D9524d1...c51
📁 0x5B13eAD53a2144...a4d


In [5]:
class TX: 
    def __init__(self, fr, to, value, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.nonce       = nonce
        self.time        = time.time()
    
    @staticmethod
    def verify(tx):
        m = encode_msg(bytes(tx))
        return w3.recover_message(m, signature=tx.sig.signature) == tx.fr
    
    def __str__ (self): return ('time:\t'   + time.ctime(self.time)+
                                '\nfrom:\t' + addr2emoji(self.fr)+' '+ph(self.fr)+
                                '\nto:\t'   + addr2emoji(self.to)+' '+ph(self.to)+
                                '\nvalue:\t'+ str(self.value)+' ether' 
                                '\nnonce:\t'+ str(self.nonce))
    def __bytes__(self): return (self.fr+
                                 self.to+
                                 str(self.value)+
                                 str(self.nonce)+
                                 str(self.time)).encode()
    
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])

In [6]:
tx1 = TX(acc1.pub, acc2.pub, 3, acc1.nonce)
tx2 = TX(acc2.pub, acc1.pub, 9, acc2.nonce)

print(tx1, '\n')
print(tx2)

time:	Thu Mar 25 19:55:02 2021
from:	🔎 0xA8Ff652D9524d1...c51
to:	📁 0x5B13eAD53a2144...a4d
value:	3 ether
nonce:	0 

time:	Thu Mar 25 19:55:02 2021
from:	📁 0x5B13eAD53a2144...a4d
to:	🔎 0xA8Ff652D9524d1...c51
value:	9 ether
nonce:	0


In [7]:
tx1 = acc1.send(acc2, 2)
assert TX.verify(tx1)

tx2 = acc2.send(acc1, 5)
assert TX.verify(tx2)

In [8]:
class Block: 
    def __init__(self, txs, prev_hash, number, time):
        self.txs       = txs
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time
    def size(self): return get_size(self)
    def __str__(self): return ('time:\t\t'        + time.ctime(self.time)+
                               '\nnumber:\t\t'    + str(self.number)+
                               '\nprev_hash:\t'   + ph(self.prev_hash)+
                               '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()
        
block = Block([tx1, tx2], rh(), 2, time.time())
print(block)

time:		Thu Mar 25 19:55:02 2021
number:		2
prev_hash:	09e25b8b1bb49f3d...866

txs:		

time:	Thu Mar 25 19:55:02 2021
from:	🔎 0xA8Ff652D9524d1...c51
to:	📁 0x5B13eAD53a2144...a4d
value:	2 ether
nonce:	0

time:	Thu Mar 25 19:55:02 2021
from:	📁 0x5B13eAD53a2144...a4d
to:	🔎 0xA8Ff652D9524d1...c51
value:	5 ether
nonce:	0



In [9]:
class MinebleBlock(Block):
    def __init__(self, 
                 txs, 
                 prev_hash, 
                 number, 
                 time, 
                 miner, 
                 diff, 
                 reward, 
                 size):
        super().__init__(txs, prev_hash, number, time)
        self.miner  = miner
        self.diff   = diff
        self.reward = reward
        self.size   = size
    def __str__(self): return (super().__str__() + 
                               '\nminer:\t\t'    + ph(self.miner)+
                               '\ndiff:\t\t'     + str(self.diff)+
                               '\nreward:\t\t'   + str(self.reward)+' ether'+
                               '\nsize:\t\t'     + str(self.size)+' bytes')
    def __bytes__(self): return (super().__bytes__().decode()+
                                 self.miner+
                                 str(self.diff)+
                                 str(self.reward)+
                                 str(self.size)).encode()
    
mblock = MinebleBlock(*unpack(block), rh(), 3, 100, block.size())
print(mblock)

time:		Thu Mar 25 19:55:02 2021
number:		2
prev_hash:	09e25b8b1bb49f3d...866

txs:		

time:	Thu Mar 25 19:55:02 2021
from:	🔎 0xA8Ff652D9524d1...c51
to:	📁 0x5B13eAD53a2144...a4d
value:	2 ether
nonce:	0

time:	Thu Mar 25 19:55:02 2021
from:	📁 0x5B13eAD53a2144...a4d
to:	🔎 0xA8Ff652D9524d1...c51
value:	5 ether
nonce:	0

miner:		f6f76821b5c0c426...f96
diff:		3
reward:		100 ether
size:		2365 bytes


In [10]:
class MinedBlock(MinebleBlock):
    def __init__(self, 
                 txs, 
                 prev_hash, 
                 number, 
                 time, 
                 miner, 
                 diff, 
                 reward, 
                 size, 
                 nonce):
        super().__init__(txs, prev_hash, number, time, miner, diff, reward, size)
        self.nonce         = nonce
        self.hash          = sha256(bytes(self)).hexdigest() 
    
    def verify_pow(self):
        candidate   = super().__bytes__()+str(self.nonce).encode()
        candidate_h = sha256(candidate).hexdigest()
        return candidate_h[:self.diff] == '0'*self.diff
    
    def __str__(self):   return (super().__str__() +
                               '\nnonce:\t\t'      + str(self.nonce)+
                               '\nhash:\t\t'       + ph(self.hash))
    def __bytes__(self): return (super().__bytes__().decode() + str(self.nonce)).encode()
    
    @staticmethod
    def create_genesis(txs): return MinedBlock(txs,'0x0',0,time.time(),'0x0',0,0,0,0)
    
mined_block = MinedBlock(*unpack(mblock), 70)
print(mined_block)

time:		Thu Mar 25 19:55:02 2021
number:		2
prev_hash:	09e25b8b1bb49f3d...866

txs:		

time:	Thu Mar 25 19:55:02 2021
from:	🔎 0xA8Ff652D9524d1...c51
to:	📁 0x5B13eAD53a2144...a4d
value:	2 ether
nonce:	0

time:	Thu Mar 25 19:55:02 2021
from:	📁 0x5B13eAD53a2144...a4d
to:	🔎 0xA8Ff652D9524d1...c51
value:	5 ether
nonce:	0

miner:		f6f76821b5c0c426...f96
diff:		3
reward:		100 ether
size:		2365 bytes
nonce:		70
hash:		b2ed53986c1f3c0b...8c4


In [11]:
mined_block = acc1.mine(block,3,100,10000)
assert mined_block.verify_pow()

In [82]:
class Blockchain:
    def __init__(self, genesis):
        self.balances,self.nonces = {},{}
        self.blocks   = []
        self.init(genesis)
        
    def init(self, genesis):
        for tx in genesis.txs: 
            self.balances[tx.to] = tx.value
            self.nonces  [tx.to] = 0
        self.blocks.append(genesis)

    def candidate(self, txs):
        return Block(txs, self.blocks[-1].hash, len(self.blocks)+1, time.time())
        
    def add(self, mb):
        assert mb.verify_pow()
        for tx in mb.txs:   assert self.apply(tx)
        assert self.verify(mb)
        self.balances[mb.miner] += mb.reward
        self.blocks.append(mb)
        return True
        
    def apply(self, tx):
        if tx.fr in self.nonces:
            assert self.nonces[tx.fr] == tx.nonce
            self.nonces[tx.fr] += 1
        else: self.nonces[tx.fr] = 0
        assert self.balances[tx.fr] - tx.value > 0
        self.balances[tx.fr] -= tx.value
        if tx.to in self.balances:   self.balances[tx.to] += tx.value
        else:                        self.balances[tx.to] = tx.value
        if tx.to not in self.nonces: self.nonces[tx.to] = 0
        return True
    
    def verify(self, mb): 
        assert mb.verify_pow()
        if len(self.blocks) > 0: assert self.blocks[-1].hash == mb.prev_hash
        for tx in mb.txs:        assert TX.verify(tx)
        return True
    
    def __str__(self):
        return '\n'.join(f'{addr2emoji(k)} {ph(k)}\nbalance: {v} \nnonce: {self.nonces[k]}\n' for k,v in self.balances.items())

In [83]:
acc1,acc2,acc3 = Account(),Account(),Account()

In [84]:
tx1 = TX('0x0', acc1.pub, 90, 0)
tx2 = TX('0x0', acc2.pub, 100, 0)

genesis = MinedBlock.create_genesis([tx1, tx2])

In [85]:
bc = Blockchain(genesis); print(bc)

💹 0x53e389876207F3...16B
balance: 90 
nonce: 0

📜 0x76021B4caD4A0F...48C
balance: 100 
nonce: 0



In [86]:
tx1 = acc1.send(acc2, 3)
tx2 = acc2.send(acc1, 9)

block = bc.candidate([tx1,tx2])
mb    = acc1.mine(block)
bc.add(mb)
print(bc)

💹 0x53e389876207F3...16B
balance: 146 
nonce: 1

📜 0x76021B4caD4A0F...48C
balance: 94 
nonce: 1



In [87]:
tx3 = acc1.send(acc3, 112)
tx4 = acc3.send(acc1, 60)

block = bc.candidate([tx3,tx4])
mb    = acc1.mine(block)
bc.add(mb)
print(bc)

💹 0x53e389876207F3...16B
balance: 144 
nonce: 2

📜 0x76021B4caD4A0F...48C
balance: 94 
nonce: 1

🔿 0xD9e45Fd7F84408...35B
balance: 52 
nonce: 1

